# Augur for Influenza A - HA protein

In [1]:
import json

## Metadata is required

Making metadata by parsing FASTA

In [1]:
from Bio import SeqIO
import re


records = []
for record in SeqIO.parse("../data/fluHA_sequences.fasta", "fasta"):
    records.append(record.description)

# Can I compile these regexes together? 

# Get strains
strainsearch = re.compile('Strain Name:(?P<strain>.*\d{2}|d{4})')

# Get ids
idsearch = re.compile('gb:(?P<id>[A-Z]+\d+[:]\d+[-]\d{4})')
# idsearch_exact = re.compile('(?P<id>gb:[A-Z]+\d+[:]\d+[-]\d{4})')

strainsList = []
idList = []
for record in records:
        i = idsearch.search(record)
#         i = idsearch_exact.search(record)
        id = i.group('id')
        idList.append(id)

        s = strainsearch.search(record)
        strain = s.group('strain')
        strainsList.append(strain)


# Get years
yearsSearch = re.compile('(?P<year>\d{4}$|\d{2}$)')
yearsList = []
for strain in strainsList:
    y = yearsSearch.search(strain)
    year = y.group('year')
    yearsList.append(year)

# Concatenate strain strings with id strings for unique identifiers
newID = []
counter = 0
for id in idList:
    newID.append(id + "|" + strainsList[counter])
    
    # Remove spaces
    if " " in newID[counter]:
        newID[counter] = newID[counter].replace(" ", "")
    counter += 1

# for id in newID:
#     print(id)
# metadata has columns:
# strain	virus	accession	date	region	country	division	city	db	segment


In [2]:
from Bio import SeqIO

original_file = r"../data/fluHA_sequences.fasta"
corrected_file = r"../data/corrected.fasta"

position = 0
with open(original_file) as original, open(corrected_file, 'w') as corrected:
    records = SeqIO.parse(original_file, 'fasta')
    for record in records:
#         print(record.id)
        record.id = newID[position]
        record.description = newID[position]
        position += 1
#         print(record.id)             
        SeqIO.write(record, corrected, 'fasta')

In [3]:
import pandas as pd

# metadata_id = list(zip(fasta_ids, strainsList, yearsList))
# metadata = list(zip(idList, strainsList, yearsList))
metadata = list(zip(newID, yearsList))

# df = pd.DataFrame(data = metadata, columns=['id', 'strain', 'year'])
df = pd.DataFrame(data = metadata, columns=['strain', 'year'])

df.to_csv('../data/metadata.tsv',sep='\t',index=False,header=True)

In [4]:
!mkdir -p ../results/

!augur filter \
  --sequences ../data/corrected.fasta \
  --metadata ../data/metadata.tsv \
  #--exclude zika-tutorial/config/dropped_strains.txt \
  --output ../results/filtered.fasta \
  --group-by year \
  --sequences-per-group 2 \
#   --min-date 1918

Traceback (most recent call last):
  File "/home/gboyle/.local/bin//augur", line 11, in <module>
    sys.exit(main())
  File "/home/gboyle/.local/lib/python3.6/site-packages/augur/__main__.py", line 10, in main
    return augur.run( argv[1:] )
  File "/home/gboyle/.local/lib/python3.6/site-packages/augur/__init__.py", line 66, in run
    return args.__command__.run(args)
  File "/home/gboyle/.local/lib/python3.6/site-packages/augur/filter.py", line 293, in run
    SeqIO.write(seq_to_keep, args.output, 'fasta')
  File "/home/gboyle/.local/lib/python3.6/site-packages/Bio/SeqIO/__init__.py", line 529, in write
    fp.write(format_function(record))
AttributeError: 'NoneType' object has no attribute 'write'


### Does pairing down the zika metadata result in the same error?

In [5]:
zika = pd.read_csv("../data/metadata-Copy1.tsv", sep='\t')

col_list = ['strain','date']
newmeta = zika[col_list]
newmeta.to_csv('../data/new-zika-metadata.tsv',sep='\t',index=False,header=True)


In [294]:
!mkdir -p ../results/

!augur filter \
  --sequences ../data/sequences-Copy1.fasta \
  --metadata ../data/new-zika-metadata.tsv \
  #--exclude zika-tutorial/config/dropped_strains.txt \
  --output ../results/filtered.fasta \
  --group-by year \
  --sequences-per-group 2 \
#   --min-date 1918

Traceback (most recent call last):
  File "/home/gboyle/.local/bin//augur", line 11, in <module>
    sys.exit(main())
  File "/home/gboyle/.local/lib/python3.6/site-packages/augur/__main__.py", line 10, in main
    return augur.run( argv[1:] )
  File "/home/gboyle/.local/lib/python3.6/site-packages/augur/__init__.py", line 66, in run
    return args.__command__.run(args)
  File "/home/gboyle/.local/lib/python3.6/site-packages/augur/filter.py", line 293, in run
    SeqIO.write(seq_to_keep, args.output, 'fasta')
  File "/home/gboyle/.local/lib/python3.6/site-packages/Bio/SeqIO/__init__.py", line 529, in write
    fp.write(format_function(record))
AttributeError: 'NoneType' object has no attribute 'write'


### How much of the metadata can we remove and still have it survive the `augur filter` command?